In [321]:
from sumpy.recurrence import _make_sympy_vec, get_processed_and_shifted_recurrence

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

import sympy as sp
import numpy as np

In [322]:
w = make_identity_diff_op(2)
laplace2d = laplacian(w)
n_init, order, r = get_processed_and_shifted_recurrence(laplace2d)

In [323]:
def scale_recurrence(r):
    #We want to subsitute s(i) r^i_{ct} = g(i)
    g = sp.Function("g")
    s = sp.Function("s")
    n = sp.symbols("n")
    rct = sp.symbols("r_{ct}")

    r_new = r*rct**n
    for i in range(order):
        r_new = r_new.subs(s(n-i),g(n-i)/(rct**(n-i)))

    return r_new

In [324]:
r_new = scale_recurrence(r)

In [325]:
def compute_derivatives(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    g_x_y = sp.log(sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2))
    derivs = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                        for i in range(p)]
    return derivs
derivs = compute_derivatives(15)

In [326]:
max_abs = .0000001
x_coord = np.random.rand()*max_abs  # noqa: NPY002
y_coord = np.random.rand()*max_abs
var = _make_sympy_vec("x", 2)
coord_dict = {var[0]: x_coord, var[1]: y_coord}

In [327]:
def evaluate_recurrence(coord_dict, rct_val, recur, p):
    subs_dict = {}
    subs_dict[g(0)] = derivs[0].subs(coord_dict).subs(rct, rct_val)
    subs_dict[g(1)] = derivs[1].subs(coord_dict).subs(rct, rct_val) * rct_val
    for i in range(2, p):
        subs_dict[g(i)] = recur.subs(n, i).subs(subs_dict).subs(coord_dict).subs(rct, rct_val)
    return np.array(list(subs_dict.values()))

In [328]:
def evaluate_true(coord_dict, rct_val, p):
    retMe = []
    for i in range(p):
        retMe.append(derivs[i].subs(coord_dict).subs(rct, rct_val)*rct_val**i)
    return np.array(retMe)

In [363]:
coord_dict = {var[0]: .0001, var[1]: 1}
exp = evaluate_recurrence(coord_dict, np.sqrt(x_coord**2 + y_coord**2), r_new, 14) 
true = evaluate_true(coord_dict, np.sqrt(x_coord**2 + y_coord**2), 14)
np.abs(exp-true)/np.abs(true)

array([0, 0, 0, 2.12051654501851e-16, 3.46279757635162e-16,
       1.22455096522596e-9, 3.67365384856080e-9, 0.0349871755639749,
       0.174935920154367, 1457799.07853295, 10204597.2972477,
       74215231574971.8, 667937431661260., 4.28164836256055e+21],
      dtype=object)

In [ ]:
coord_dict = {var[0]: .0001, var[1]: 1}
exp = evaluate_recurrence(coord_dict, 1, r_new, 14) 
true = evaluate_true(coord_dict, 1, 14)
np.abs(exp-true)/np.abs(true)

array([0, 0, 3.63907034626755e-16, 0, 3.97761921240578e-16,
       2.30632677901825e-13, 6.92729901941019e-13, 6.59135170193698e-10,
       3.30367124192758e-9, 2.74821958755983e-6, 1.93084640776258e-5,
       0.0140018021588909, 0.126675562113591, 80.8527473439207],
      dtype=object)